I've used this notebook to derive the BP algorithm on top of the PGMPY library, whose BP isn't a real BP algorithm

In [42]:
import src.models.builders as mb
import src.inference.helpers as ih

In [43]:
# Create factor graph
height = 170
age = 30
sex = "Male"
model, inf_alg, HFEV1, ecFEV1, AR, HO2Sat, O2SatFFA, IA, UO2Sat, O2Sat = (
    mb.o2sat_fev1_point_in_time_model_2(height, age, sex)
)

buidling custom model (work in progress)


# BP message computation

Add this to the FactorGraph library

def point_mass_message(self, var, obs):
    """
    Returns the point mass message for the variable given the observed state.
    """
    card = self.get_cardinality(var)
    # Create an array with 1 at the index of the evidence and 0 elsewhere
    message = np.zeros(card)
    message[obs] = 1
    return message

In [44]:
# Functions moved to the custom BP algorithm
# def variable_node_message(incoming_messages):
#     """
#     The outgoing message is the element wise product of all incoming messages
#     """
#     if len(incoming_messages) == 1:
#         return incoming_messages[0]
#     outgoing_message = np.multiply(*incoming_messages)
#     # Normalise
#     return outgoing_message / np.sum(outgoing_message)


# def factor_node_message(incoming_messages, factor, target_var):
#     """
#     Returns the outgoing message for a factor node, which is the multiplication of the incoming messages with the factor function (CPT)

#     The variables' order in the incoming messages list must match the variable's order in the CPT's dimensions
#     """
#     cpt = factor.values
#     vars = factor.variables

#     assert (
#         len(incoming_messages) == cpt.ndim - 1
#     ), f"Error computing factor node message for {target_var}. The number of incoming messages must equal the card(CPT) - 1"

#     # Ensure that the target var is on the CPT's 0th axis
#     # Find idx of target var in the vars list
#     target_var_idx = vars.index(target_var)
#     if target_var_idx != 0:
#         # Move target var to the 0th axis
#         cpt = np.moveaxis(cpt, target_var_idx, 0)

#     # Invert incoming_messages, so that the first message corresponds to the last dimension of the CPT
#     incoming_messages = list(reversed(incoming_messages))

#     # Reduce the CPT with the inverted list of incoming messages
#     outgoing_message = reduce(
#         lambda cpt_reduced, m: np.matmul(cpt_reduced, m), incoming_messages, cpt
#     )

#     # Normalise
#     return outgoing_message / sum(outgoing_message)

In [45]:
# Test implementation
var_to_infer = "Healthy FEV1 (L)"

ecFEV1_obs = 1
ecFEV1_idx = ecFEV1.get_bin_for_value(ecFEV1_obs)[1]

o2sat_obs = 100
o2sat_idx = O2Sat.get_bin_for_value(o2sat_obs)[1]

hfev1_factor = model.get_factors()[0]
ecfev1_factor = model.get_factors()[1]
ar_factor = model.get_factors()[2]
ho2sat_factor = model.get_factors()[3]
o2satffa_factor = model.get_factors()[4]
ia_factor = model.get_factors()[5]
uo2sat_factor = model.get_factors()[6]
o2sat_factor = model.get_factors()[7]

# O2sat -> O2sat factor
o2sat_obs_m = model.point_mass_message(O2Sat.name, o2sat_idx)
o2sat_up = inf_alg.variable_node_message([o2sat_obs_m])

# Only one message comes to UO2Sat
to_uo2sat = inf_alg.factor_node_message([o2sat_up], o2sat_factor, UO2Sat.name)
# Only one message incoming UO2Sat, hence
uo2sat_up = inf_alg.variable_node_message([to_uo2sat])
assert (uo2sat_up == to_uo2sat).all()

# IA -> O2Sat factor
ia_right = inf_alg.variable_node_message([IA.cpt])

# uo2sat_factor -> o2satffa
to_o2satffa = inf_alg.factor_node_message(
    [uo2sat_up, ia_right], uo2sat_factor, O2SatFFA.name
)

# O2SatFFA to O2SatFFA factor
o2satffa_up = inf_alg.variable_node_message([to_o2satffa])
assert (o2satffa_up == to_o2satffa).all()

# HO2Sat to O2SatFFA factor
ho2sat_down = inf_alg.variable_node_message([HO2Sat.cpt])

# O2SatFFA factor to AR
to_ar = inf_alg.factor_node_message(
    [o2satffa_up, ho2sat_down], o2satffa_factor, AR.name
)

# AR -> ecfev1 factor
ar_left = inf_alg.variable_node_message([AR.cpt, to_ar])

# ecFEV1 -> ecfev1 factor
ecfev1_obs_m = model.point_mass_message(ecFEV1.name, ecFEV1_idx)
ecfev1_up = inf_alg.variable_node_message([ecfev1_obs_m])

# ecfev1 -> hfev1
to_hfev1 = inf_alg.factor_node_message(
    [ecfev1_up, ar_left], model.get_factors()[1], var_to_infer
)

# hfev1 posterior
hfev1_posterior = inf_alg.variable_node_message([to_hfev1, HFEV1.cpt])
hfev1_posterior.shape

(100,)

In [52]:
# Compare results against the bayes net model (factor graph model takes ages to compute)
model1, inf_alg1, _, _, _, _, _, _, _, _ = mb.o2sat_fev1_point_in_time_model(
    height, age, sex
)
res = ih.infer(inf_alg1, [HFEV1], [[ecFEV1, ecFEV1_obs], [O2Sat, o2sat_obs]])
res[HFEV1.name].values.shape

((hfev1_posterior - res[HFEV1.name].values) < 1e-5).all()
# Plot the two posteriors with plotly go figure
import plotly.graph_objs as go

fig = go.Figure()
# Add bar plot for the factor graph model
fig.add_trace(go.Bar(x=HFEV1.bins, y=hfev1_posterior, name="Factor Graph with custom<br>BP computation"))
fig.add_trace(go.Bar(x=HFEV1.bins, y=res[HFEV1.name].values, name="Bayes Net"))
# Update layout
fig.update_layout(
    barmode="group", title="Comparison of HFEV1 posteriors", width=800, height=300
)
fig.show()

In [6]:
((hfev1_posterior - res[HFEV1.name].values) < 1e-5).all()
# Plot the two posteriors with plotly go figure
import plotly.graph_objs as go

fig = go.Figure()
# Add bar plot for the factor graph model
fig.add_trace(go.Bar(x=HFEV1.bins, y=hfev1_posterior, name="Factor Graph"))
fig.add_trace(go.Bar(x=HFEV1.bins, y=res[HFEV1.name].values, name="Bayes Net"))
# Update layout
fig.update_layout(
    barmode="group", title="Comparison of HFEV1 posteriors", width=800, height=300
)
fig.show()

# BP scheduling with recursion

In [5]:
var_to_infer = "Healthy FEV1 (L)"

ecFEV1_obs = 1
ecFEV1_idx = ecFEV1.get_bin_for_value(ecFEV1_obs)[1]
o2sat_obs = 100
o2sat_idx = O2Sat.get_bin_for_value(o2sat_obs)[1]

evidences = {ecFEV1.name: ecFEV1_idx, O2Sat.name: o2sat_idx}
evidences

obs_vars = evidences.keys()

In [36]:
# Returns the posterior distribution of the queried variable, recursively going through the graph until a node stopping criteria is met.


def query(model, inf_alg, var, evidences):
    """
    Returns the posterior distribution of the queried variable, recursively going through the graph until reaching a root variable (with no parent), or an observed variable.
    """
    return process_var(model, inf_alg, evidences, var)


def process_var(model, inf_alg, evidences, var, from_factor=None, debug=False):
    """
    Returns the message outgoing from the variable node, given the incoming messages from its neighbouring factors.

    evidences: dict with the observed variables and their values
    var: str, the variable from which we want to compute the outgoing message
    from_factor: str, the factor asking to process that variable, as part of the recursion.
    from_factor is None for the first call, i.e. for the queried variable from which we want to compute the posterior.
    """
    print(f"Processing variable {var}")
    # Stopping criteria: if the variable is observed, return the point mass message of the observation
    if var in evidences.keys():
        print(f"Returning point mass message for {var}")
        return model.point_mass_message(var, evidences[var])
    # Else, get the incoming messages from all neighbouring factors
    else:
        print(f"Variable {var} has multiple connections, going deeper in the graph")
        incoming_messages = []
        for factor in model.neighbors(var):
            if factor != from_factor:
                incoming_messages.append(
                    process_factor(model, inf_alg, evidences, factor, from_var=var)
                )
        print(f"Computing outgoing message for the {var}\n")
        return inf_alg.variable_node_message(incoming_messages)


def process_factor(model, inf_alg, evidences, factor, from_var: str):
    """
    Returns the message outgoing from the factor node, given the incoming messages from its neighbouring variables.

    factor: str, the factor from which we want to compute the outgoing message
    from_var: str, the variable asking to process that factor, as part of the recursion.
    from_var is None for the first call, i.e. for the queried variable from which we want to compute the posterior.
    """
    print(f"Processing a factor of {from_var}")
    # from_var can't be null
    assert from_var is not None, "from_var must be specified"

    vars = factor.variables
    # Stopping criteria: if the factor is connected to only one variable, return the factor function which is the prior of from_var
    if len(vars) == 1:
        print(f"The factor is {from_var}'s prior, returning it as the message")
        prior = factor.values
        assert prior.ndim == 1, "The factor function must be a 1D array"
        return prior
    # Else, get the incoming messages from all neighbouring variables
    else:
        print(
            f"This {from_var}'s factor has multiple connections, going deeper in the graph"
        )
        incoming_messages = []
        for var in vars:
            if var != from_var:
                incoming_messages.append(
                    process_var(model, inf_alg, evidences, var, from_factor=factor)
                )
        return inf_alg.factor_node_message(incoming_messages, factor, from_var)

In [47]:
# Test implementation
hfev1_posterior_recursive = query(model, inf_alg, HFEV1.name, evidences)

Processing variable Healthy FEV1 (L)
Variable Healthy FEV1 (L) has multiple connections, going deeper in the graph
Processing a factor of Healthy FEV1 (L)
The factor is Healthy FEV1 (L)'s prior, returning it as the message
Processing a factor of Healthy FEV1 (L)
This Healthy FEV1 (L)'s factor has multiple connections, going deeper in the graph
Processing variable ecFEV1 (L)
Returning point mass message for ecFEV1 (L)
Processing variable Airway resistance (%)
Variable Airway resistance (%) has multiple connections, going deeper in the graph
Processing a factor of Airway resistance (%)
The factor is Airway resistance (%)'s prior, returning it as the message
Processing a factor of Airway resistance (%)
This Airway resistance (%)'s factor has multiple connections, going deeper in the graph
Processing variable O2 saturation if fully functional alveoli (%)
Variable O2 saturation if fully functional alveoli (%) has multiple connections, going deeper in the graph
Processing a factor of O2 satu

In [51]:
# Compare results against the bayes net model (factor graph model takes ages to compute)
model1, inf_alg1, _, _, _, _, _, _, _, _ = mb.o2sat_fev1_point_in_time_model(
    height, age, sex
)
res = ih.infer(inf_alg1, [HFEV1], [[ecFEV1, ecFEV1_obs], [O2Sat, o2sat_obs]])
res[HFEV1.name].values.shape

((hfev1_posterior_recursive - res[HFEV1.name].values) < 1e-5).all()
# Plot the two posteriors with plotly go figure
import plotly.graph_objs as go

fig = go.Figure()
# Add bar plot for the factor graph model
fig.add_trace(go.Bar(x=HFEV1.bins, y=hfev1_posterior_recursive, name="Factor Graph with custom<br>BP computation and scheduling"))
fig.add_trace(go.Bar(x=HFEV1.bins, y=res[HFEV1.name].values, name="Bayes Net"))
# Update layout
fig.update_layout(
    barmode="group", title="Comparison of HFEV1 posteriors", width=800, height=300
)
fig.show()